In [1]:
!pip install datasets

     |████████████████████████████████| 510 kB 3.6 MB/s            
     |████████████████████████████████| 38.3 MB 34.9 MB/s            
     |████████████████████████████████| 170 kB 37.9 MB/s            
     |████████████████████████████████| 193 kB 37.9 MB/s            
     |████████████████████████████████| 388 kB 37.3 MB/s            
     |████████████████████████████████| 133 kB 38.3 MB/s            
     |████████████████████████████████| 116 kB 40.1 MB/s            
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2021.11.1
    Uninstalling fsspec-2021.11.1:
      Successfully uninstalled fsspec-2021.11.1
  Attempting uninstall: dill
    Found existing installation: dill 0.3.4
    Uninstalling dill-0.3.4:
      Successfully uninstalled dill-0.3.4


In [2]:
!pip install transformers

     |████████████████████████████████| 8.8 MB 3.6 MB/s            
     |████████████████████████████████| 1.3 MB 79.9 MB/s            
     |████████████████████████████████| 773 kB 87.2 MB/s            
     |████████████████████████████████| 3.6 MB 73.3 MB/s            


In [3]:
import torch

In [4]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModel, BertTokenizer

In [5]:
#device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = torch.device('cpu') #computed the embeddings on the VM CPU because the GPU didn't have enough memory

In [6]:
dataset = load_dataset("stanfordnlp/sst2", download_mode="force_redownload")

Generating train split:   0%|          | 0/67349 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/872 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1821 [00:00<?, ? examples/s]

In [7]:
dataset = dataset['train'].train_test_split(test_size=0.15, seed=33)
dataset

DatasetDict({
    train: Dataset({
        features: ['idx', 'sentence', 'label'],
        num_rows: 57246
    })
    test: Dataset({
        features: ['idx', 'sentence', 'label'],
        num_rows: 10103
    })
})

In [8]:
import os
cdir = os.getcwd()
dataset.save_to_disk(str(cdir) + "/data")

Saving the dataset (0/1 shards):   0%|          | 0/57246 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/10103 [00:00<?, ? examples/s]

In [9]:
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
model = AutoModel.from_pretrained("distilbert-base-uncased").to(device)

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

In [10]:
train_x = dataset['train']['sentence']
tokenized_train_x = tokenizer(train_x, padding = True, truncation = True, return_tensors = "pt") 

In [11]:
test_x = dataset['test']['sentence']
tokenized_test_x = tokenizer(test_x, padding = True, truncation = True, return_tensors = "pt")

In [12]:
tokenized_train_x = {k:torch.tensor(v).to(device) for k,v in tokenized_train_x.items()}
tokenized_test_x = {k:torch.tensor(v).to(device) for k,v in tokenized_test_x.items()}

/tmp/ipykernel_106089/355028685.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  tokenized_train_x = {k:torch.tensor(v).to(device) for k,v in tokenized_train_x.items()}
/tmp/ipykernel_106089/355028685.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  tokenized_test_x = {k:torch.tensor(v).to(device) for k,v in tokenized_test_x.items()}


In [13]:
#This takes a while to run but it works on my VM
with torch.no_grad():
  hidden_train_x = model(**tokenized_train_x)
  hidden_test_x = model(**tokenized_test_x)

In [14]:
cls_train_x = hidden_train_x.last_hidden_state[:,0,:]
cls_test_x = hidden_test_x.last_hidden_state[:,0,:]

In [15]:
torch.save(cls_train_x, 'x_train_tensors.pt') 
torch.save(cls_test_x, 'x_test_tensors.pt')